---
# Processing system
---

In [2]:
import os.path
import numpy as np
import pandas as pd
import time
from datetime import datetime

import shtReader_py.shtRipper as shtRipper
from source.Files_operating import read_dataFile #, read_sht_dalpha
from source.NN_environment import process_fragments, get_borders, normalise_series, down_to_zero
from source.NN_environment import get_prediction_unet

In [7]:
def read_sht_dalpha(filename: str, filepath: str) -> pd.DataFrame:
    if ".sht" in filename.lower():
        res = shtRipper.ripper.read(filepath + filename)
    else:
        res = shtRipper.ripper.read(filepath + filename + ".SHT")

    data = np.array([res["D-alfa  хорда R=50 cm"]["x"], res["D-alfa  хорда R=50 cm"]["y"]])
    dalpha_df = pd.DataFrame(data.transpose(), columns=["t", "D-alpha_h50"])
    return dalpha_df

In [12]:
interval_path = "data/intervals/"
signal_path = "data/d-alpha/"

filename = "sht44351"
filepath = "C:/Users/f.belous/Work/Projects/D-alpha-instability-search/data/sht/"
F_ID = filename[-5:]

In [ ]:
df = read_sht_dalpha(filename, filepath)  # pd.read_csv(signal_path + filename + ".txt", sep=',')  # read_dataFile(interval_path + filename + "_exportGlobus2.dat", F_ID)
df = df.rename(columns={"D-alpha_h50": "ch1"})
df["ch1_marked"] = 0
df["ch1_ai_marked"] = 0

df.head()

In [9]:
start_time = time.time()

ckpt_v=4
df["ch1_ai_marked"] = get_prediction_unet(df["ch1"].to_numpy(), ckpt_v=ckpt_v)  # , old=True

df["ch1_marked"] = down_to_zero(np.array(df["ch1_ai_marked"]), edge=0.5)
df["ch1_marked"] = process_fragments(np.array(df["ch1"]), np.array(df["ch1_marked"]), length_edge=30, scale=0)  # old version: length_edge=20

to_pack = {
    "D-alpha, chord=50 cm": {
        'comment': f'SHOT: #{F_ID}',
        'unit': 'U(V)',
        # 'x': df.t,
        'tMin': df.t.min(),  # minimum time
        'tMax': df.t.max(),  # maximum time
        'offset': 0.0,  # ADC zero level offset
        'yRes': 0.001,  # ADC resolution: 0.0001 Volt per adc bit
        'y': df.ch1.to_list()
    },
    "Mark": {
        'comment': f'ELMs marks (by proc-sys v2.0-0scl; {datetime.now().strftime("%d.%m.%Y")})',
        'unit': 'U(V)',
        # 'x': df.t,
        'tMin': df.t.min(),  # minimum time
        'tMax': df.t.max(),  # maximum time
        'offset': 0.0,  # ADC zero level offset
        'yRes': 0.001,  # ADC resolution: 0.0001 Volt per adc bit
        'y': df.ch1_marked.to_list()
    },
    "AI prediction": {
        'comment': f'Processed NN prediction of ELMs (v{ckpt_v}; trn-on: #44[168|184|194] )',
        'unit': 'U(V)',
        # 'x': df.t,
        'tMin': df.t.min(),  # minimum time
        'tMax': df.t.max(),  # maximum time
        'offset': 0.0,  # ADC zero level offset
        'yRes': 0.001,  # ADC resolution: 0.0001 Volt per adc bit
        'y': df.ch1_ai_marked.to_list()
    },
}

packed = shtRipper.ripper.write(path=filepath + "marked/", filename=f'{F_ID}_data.SHT', data=to_pack)

print(f"Result saved successfully to {filepath}marked/{F_ID}_data.SHT")
print(f"Took - {round(time.time() - start_time, 2)} s")

C:\Users\f.belous\AppData\Local\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 52 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


requested path "44351_data.SHT" does not exist.
Result saved successfully to D:/Edu/Lab/D-alpha-instability-search/data/sht/marked/44351_data.SHT
Took - 440.15 s


In [10]:
df.to_csv('data/d-alpha/' + f"df/{F_ID}_data.csv", index=False)